In [ ]:
# =============================================================================
# PROJE: Qwen 2-VL Beslenme Asistanı (Architect: Halit)
# DOSYA: FET312_23040301091_GYMPT_1.ipynb
# AMAÇ:  Modeli yüklemek, GPU kontrolü yapmak
# =============================================================================

In [ ]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor,BitsAndBytesConfig
from PIL import Image
import requests
from io import BytesIO

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Çalışma Ortamı: {device.upper()}")
if device == "cuda":
    print(f"GPU Modeli: {torch.cuda.get_device_name(0)}")
    print(f"VRAM Bilgisi: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("UYARI: CPU kullanıyorsunuz. Model çok yavaş çalışacaktır!")

Çalışma Ortamı: CUDA
GPU Modeli: Tesla T4
VRAM Bilgisi: 15.83 GB


In [ ]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    dtype=torch.bfloat16,   # torch_dtype değil artık dtype
    device_map="auto",
    attn_implementation="sdpa"  # flash_attention kapalı, sdpa stabil ve hızlı
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [15]:

question = "Bana kahvaltı için 1000 kalorilik bir menü önerir misin?"

# 1- Chat Formatı için dönüştürme
text_prompt = processor.apply_chat_template(
    [{"role": "user", "content": question}],
    tokenize=False,
    add_generation_prompt=True
)

# 2- Modele Ver (Resim yoksa images=None)
inputs = processor(
    text=[text_prompt],
    images=None,
    return_tensors="pt",
    padding=True
).to(model.device)

# 3- Cevabı Al
output = model.generate(**inputs, max_new_tokens=100)
answer = processor.batch_decode(output, skip_special_tokens=True)[0]

# Cevabı temizle
print(answer.split("assistant")[-1].strip())

Tabii ki, size bir kahvaltı menüsü öneririm. Ancak, size daha fazla bilgi vermek için size hangi türde bir kahvaltı yapmak istersiniz? (örneğin, sağlıklı, lezzetli, vb.)


In [18]:
import requests
from PIL import Image
from io import BytesIO

# 1. Resmi Al
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/0/0b/RedDot_Burger.jpg/800px-RedDot_Burger.jpg"
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)
image = Image.open(BytesIO(response.content))

# 2. PROMPT'U (Modeli base model için kırmanın yolu)
prompt = """
Bu bir görsel analiz ve nesne tanıma testidir.
Fotoğraftaki yiyecekleri tespit et (Hamburger, patates vb.).
Her bir yiyecek için genel besin veritabanlarında (örneğin USDA) kabul edilen ortalama kalori değerlerini yaz.
Kişisel tavsiye verme, sadece nesneleri ve standart kalori değerlerini listele.
Sonuçları şu formatta ver:
- Yiyecek Adı: Tahmini Kalori
- Toplam: Tahmini Toplam
"""

conversation = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": prompt},
        ],
    }
]

# 3. İşleme
text_prompt = processor.apply_chat_template(conversation, tokenize=False, add_generation_prompt=True)
inputs = processor(text=[text_prompt], images=[image], return_tensors="pt", padding=True).to("cuda")

# 4. Cevabı Al
output = model.generate(**inputs, max_new_tokens=256)
print(processor.batch_decode(output, skip_special_tokens=True)[0].split("assistant")[-1])


- Yiyecek Adı: Hamburger
  - Tahmini Kalori: 500-600 kalori
- Yiyecek Adı: Patates
  - Tahmini Kalori: 150-200 kalori
- Toplam: Tahmini Toplam
  - Tahmini Kalori: 650-700 kalori
